In [ ]:
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）

In [ ]:
# ここから！
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    for i in range(len(X) - time_steps + 1):
        X_reshaped.append(X[i:i + time_steps])
    return np.array(X_reshaped)

# データの整形
time_steps = 5
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]

# データの分割（80%を学習、20%を評価）
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# データの形状確認
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

# データをPyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# データセットとデータローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# モデルのインスタンスを作成
input_size = X_train_tensor.shape[2]
hidden_size = 50
output_size = 3  # アクセントのクラス数（0, 1, 2）

model = LSTMModel(input_size, hidden_size, output_size)

# 損失関数とオプティマイザの定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 評価結果を保存するリスト
val_losses = []
val_accuracies = []

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

# 学習
num_epochs = 50
epoch_losses = []  # 各エポックの平均損失を格納するリスト

for epoch in range(num_epochs):
    model.train()  # モデルをトレーニングモードに設定
    epoch_loss = 0
    
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    average_loss = epoch_loss / len(train_dataloader)
    epoch_losses.append(average_loss)  # 平均損失をリストに追加
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}')

# 損失の折れ線グラフを描画
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.xticks(range(1, num_epochs + 1))  # x軸の目盛りを設定
plt.grid()
plt.show()

In [ ]:
# エポック数：100
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    for i in range(len(X) - time_steps + 1):
        X_reshaped.append(X[i:i + time_steps])
    return np.array(X_reshaped)

# データの整形
time_steps = 5
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]

# データの分割（80%を学習、20%を評価）
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# データの形状確認
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

# データをPyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# データセットとデータローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# モデルのインスタンスを作成
input_size = X_train_tensor.shape[2]
hidden_size = 50
output_size = 3  # アクセントのクラス数（0, 1, 2）

model = LSTMModel(input_size, hidden_size, output_size)

# 損失関数とオプティマイザの定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 評価結果を保存するリスト
val_losses = []
val_accuracies = []

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

# 学習
num_epochs = 100
epoch_losses = []  # 各エポックの平均損失を格納するリスト

for epoch in range(num_epochs):
    model.train()  # モデルをトレーニングモードに設定
    epoch_loss = 0
    
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    average_loss = epoch_loss / len(train_dataloader)
    epoch_losses.append(average_loss)  # 平均損失をリストに追加
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}')

# 損失の折れ線グラフを描画
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.xticks(range(1, num_epochs + 1))  # x軸の目盛りを設定
plt.grid()
plt.show()

In [ ]:
# ここから
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    
    for i in range(len(X) - time_steps + 1):  # 修正: +1を追加
        X_reshaped.append(X[i:i + time_steps])
        
    return np.array(X_reshaped)

# データの整形
time_steps = 5  # 5つの時間ステップを使用
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]  # 最初の4つはシーケンスが足りないためカット

# データの形状確認
print(X_reshaped.shape, y_reshaped.shape)

In [ ]:
# エポック数；120
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    for i in range(len(X) - time_steps + 1):
        X_reshaped.append(X[i:i + time_steps])
    return np.array(X_reshaped)

# データの整形
time_steps = 5
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]

# データの分割（80%を学習、20%を評価）
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# データの形状確認
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

# データをPyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# データセットとデータローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# モデルのインスタンスを作成
input_size = X_train_tensor.shape[2]
hidden_size = 50
output_size = 3  # アクセントのクラス数（0, 1, 2）

model = LSTMModel(input_size, hidden_size, output_size)

# 損失関数とオプティマイザの定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 評価結果を保存するリスト
val_losses = []
val_accuracies = []

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

# 学習
num_epochs = 120
epoch_losses = []  # 各エポックの平均損失を格納するリスト

for epoch in range(num_epochs):
    model.train()  # モデルをトレーニングモードに設定
    epoch_loss = 0
    
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    average_loss = epoch_loss / len(train_dataloader)
    epoch_losses.append(average_loss)  # 平均損失をリストに追加
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}')

# 損失の折れ線グラフを描画
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.xticks(range(1, num_epochs + 1))  # x軸の目盛りを設定
plt.grid()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    
    for i in range(len(X) - time_steps + 1):  # 修正: +1を追加
        X_reshaped.append(X[i:i + time_steps])
        
    return np.array(X_reshaped)

# データの整形
time_steps = 5  # 5つの時間ステップを使用
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]  # 最初の4つはシーケンスが足りないためカット

# データの形状確認
print(X_reshaped.shape, y_reshaped.shape)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    for i in range(len(X) - time_steps + 1):
        X_reshaped.append(X[i:i + time_steps])
    return np.array(X_reshaped)

# データの整形
time_steps = 5
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]

# データの分割（80%を学習、20%を評価）
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# データの形状確認
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

# データをPyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# データセットとデータローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# モデルのインスタンスを作成
input_size = X_train_tensor.shape[2]
hidden_size = 100
output_size = 3  # アクセントのクラス数（0, 1, 2）

model = LSTMModel(input_size, hidden_size, output_size)

# 損失関数とオプティマイザの定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 評価結果を保存するリスト
val_losses = []
val_accuracies = []

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

# 学習
num_epochs = 100
epoch_losses = []  # 各エポックの平均損失を格納するリスト

for epoch in range(num_epochs):
    model.train()  # モデルをトレーニングモードに設定
    epoch_loss = 0
    
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    average_loss = epoch_loss / len(train_dataloader)
    epoch_losses.append(average_loss)  # 平均損失をリストに追加
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}')

# 損失の折れ線グラフを描画
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.xticks(range(1, num_epochs + 1))  # x軸の目盛りを設定
plt.grid()
plt.show()

In [ ]:
# 双方向LSTM+２つのドロップアウト層
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    
    for i in range(len(X) - time_steps + 1):  # 修正: +1を追加
        X_reshaped.append(X[i:i + time_steps])
        
    return np.array(X_reshaped)

# データの整形
time_steps = 5  # 5つの時間ステップを使用
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]  # 最初の4つはシーケンスが足りないためカット

# データの形状確認
print(X_reshaped.shape, y_reshaped.shape)

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    for i in range(len(X) - time_steps + 1):
        X_reshaped.append(X[i:i + time_steps])
    return np.array(X_reshaped)

# データの整形
time_steps = 5
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]

# データの分割（80%を学習、20%を評価）
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# データの形状確認
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

# データをPyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# データセットとデータローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# BiLSTMモデルの定義
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout_prob=0.5):
        super(BiLSTMModel, self).__init__()
        # 2つの双方向LSTM層を設定
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, 
                            dropout=dropout_prob, bidirectional=True, batch_first=True)
        # ドロップアウト層を設定
        self.dropout = nn.Dropout(dropout_prob)
        # 双方向LSTMなのでhidden_size * 2を次元に持つ全結合層
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        # LSTM層を通過
        lstm_out, (hn, cn) = self.lstm(x)
        # ドロップアウトを適用
        lstm_out = self.dropout(lstm_out)
        # 双方向なので、順方向と逆方向の出力を結合 (hidden statesの最後を使用)
        hn_concat = torch.cat((hn[-2], hn[-1]), dim=1)  # 双方向の最終層の出力を結合
        # 全結合層に通して出力
        out = self.fc(hn_concat)
        return out

# モデルのインスタンスを作成
input_size = X_train_tensor.shape[2]
hidden_size = 50
output_size = 3  # アクセントのクラス数（0, 1, 2）
num_layers = 2
dropout_prob = 0.5

model = BiLSTMModel(input_size, hidden_size, output_size, num_layers=num_layers, dropout_prob=dropout_prob)

# 損失関数とオプティマイザの定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 評価結果を保存するリスト
val_losses = []
val_accuracies = []

import matplotlib.pyplot as plt
from tqdm import tqdm

# 学習
num_epochs = 100
epoch_losses = []  # 各エポックの平均損失を格納するリスト

for epoch in range(num_epochs):
    model.train()  # モデルをトレーニングモードに設定
    epoch_loss = 0
    
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    average_loss = epoch_loss / len(train_dataloader)
    epoch_losses.append(average_loss)  # 平均損失をリストに追加
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}')

# 損失の折れ線グラフを描画
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.xticks(range(1, num_epochs + 1))  # x軸の目盛りを設定
plt.grid()
plt.show()

In [ ]:
# 双方向LSTM+２つのドロップアウト層
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    
    for i in range(len(X) - time_steps + 1):  # 修正: +1を追加
        X_reshaped.append(X[i:i + time_steps])
        
    return np.array(X_reshaped)

# データの整形
time_steps = 5  # 5つの時間ステップを使用
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]  # 最初の4つはシーケンスが足りないためカット

# データの形状確認
print(X_reshaped.shape, y_reshaped.shape)

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    for i in range(len(X) - time_steps + 1):
        X_reshaped.append(X[i:i + time_steps])
    return np.array(X_reshaped)

# データの整形
time_steps = 5
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]

# データの分割（80%を学習、20%を評価）
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# データの形状確認
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

# データをPyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# データセットとデータローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# BiLSTMモデルの定義
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout_prob=0.5):
        super(BiLSTMModel, self).__init__()
        # 2つの双方向LSTM層を設定
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, 
                            dropout=dropout_prob, bidirectional=True, batch_first=True)
        # ドロップアウト層を設定
        self.dropout = nn.Dropout(dropout_prob)
        # 双方向LSTMなのでhidden_size * 2を次元に持つ全結合層
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        # LSTM層を通過
        lstm_out, (hn, cn) = self.lstm(x)
        # ドロップアウトを適用
        lstm_out = self.dropout(lstm_out)
        # 双方向なので、順方向と逆方向の出力を結合 (hidden statesの最後を使用)
        hn_concat = torch.cat((hn[-2], hn[-1]), dim=1)  # 双方向の最終層の出力を結合
        # 全結合層に通して出力
        out = self.fc(hn_concat)
        return out

# モデルのインスタンスを作成
input_size = X_train_tensor.shape[2]
hidden_size = 50
output_size = 3  # アクセントのクラス数（0, 1, 2）
num_layers = 2
dropout_prob = 0.5

model = BiLSTMModel(input_size, hidden_size, output_size, num_layers=num_layers, dropout_prob=dropout_prob)

# 損失関数とオプティマイザの定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 評価結果を保存するリスト
val_losses = []
val_accuracies = []

import matplotlib.pyplot as plt
from tqdm import tqdm

# 学習
num_epochs = 120
epoch_losses = []  # 各エポックの平均損失を格納するリスト

for epoch in range(num_epochs):
    model.train()  # モデルをトレーニングモードに設定
    epoch_loss = 0
    
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    average_loss = epoch_loss / len(train_dataloader)
    epoch_losses.append(average_loss)  # 平均損失をリストに追加
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}')

# 損失の折れ線グラフを描画
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.xticks(range(1, num_epochs + 1))  # x軸の目盛りを設定
plt.grid()
plt.show()

In [ ]:
# 双方向LSTM+２つのドロップアウト層
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')  # 特徴量とラベルが一緒のデータ

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    
    for i in range(len(X) - time_steps + 1):  # 修正: +1を追加
        X_reshaped.append(X[i:i + time_steps])
        
    return np.array(X_reshaped)

# データの整形
time_steps = 5  # 5つの時間ステップを使用
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]  # 最初の4つはシーケンスが足りないためカット

# データの形状確認
print(X_reshaped.shape, y_reshaped.shape)

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# reshape_data関数を定義
def reshape_data(X, time_steps=1):
    X_reshaped = []
    for i in range(len(X) - time_steps + 1):
        X_reshaped.append(X[i:i + time_steps])
    return np.array(X_reshaped)

# データの整形
time_steps = 5
X_reshaped = reshape_data(X, time_steps=time_steps)
y_reshaped = y[time_steps - 1:]

# データの分割（80%を学習、20%を評価）
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# データの形状確認
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

# データをPyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# データセットとデータローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# BiLSTMモデルの定義
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout_prob=0.5):
        super(BiLSTMModel, self).__init__()
        # 2つの双方向LSTM層を設定
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, 
                            dropout=dropout_prob, bidirectional=True, batch_first=True)
        # ドロップアウト層を設定
        self.dropout = nn.Dropout(dropout_prob)
        # 双方向LSTMなのでhidden_size * 2を次元に持つ全結合層
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        # LSTM層を通過
        lstm_out, (hn, cn) = self.lstm(x)
        # ドロップアウトを適用
        lstm_out = self.dropout(lstm_out)
        # 双方向なので、順方向と逆方向の出力を結合 (hidden statesの最後を使用)
        hn_concat = torch.cat((hn[-2], hn[-1]), dim=1)  # 双方向の最終層の出力を結合
        # 全結合層に通して出力
        out = self.fc(hn_concat)
        return out

# モデルのインスタンスを作成
input_size = X_train_tensor.shape[2]
hidden_size = 50
output_size = 3  # アクセントのクラス数（0, 1, 2）
num_layers = 2
dropout_prob = 0.5

model = BiLSTMModel(input_size, hidden_size, output_size, num_layers=num_layers, dropout_prob=dropout_prob)

# 損失関数とオプティマイザの定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 評価結果を保存するリスト
val_losses = []
val_accuracies = []

import matplotlib.pyplot as plt
from tqdm import tqdm

# 学習
num_epochs = 150
epoch_losses = []  # 各エポックの平均損失を格納するリスト

for epoch in range(num_epochs):
    model.train()  # モデルをトレーニングモードに設定
    epoch_loss = 0
    
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
    
    average_loss = epoch_loss / len(train_dataloader)
    epoch_losses.append(average_loss)  # 平均損失をリストに追加
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}')

# 損失の折れ線グラフを描画
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), epoch_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.xticks(range(1, num_epochs + 1))  # x軸の目盛りを設定
plt.grid()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）
print(X)
print(y)

In [ ]:
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）

# 特徴量をDataFrameに変換してCSVファイルとして保存
X_df = pd.DataFrame(X, columns=['Mean', 'Max', 'Min', 'Std', 'Median'])
X_df.to_csv('output_features.csv', index=False)  # index=Falseでインデックスを保存しない

In [ ]:
# 各音声の区切りがわからない
import pandas as pd
import numpy as np
import glob

def load_and_preprocess_data(folder_path):
    # f0_statistics_1.csv から f0_statistics_5000.csv までのファイルを指定して読み込む
    csv_files = [f"{folder_path}/f0_statistics_{i}.csv" for i in range(1, 5001)]
    
    data_list = []
    
    for file in csv_files:
        df = pd.read_csv(file)
        # 必要な列を取得
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        data_list.append(data)
    
    return np.concatenate(data_list, axis=0)

# データのロード
data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルを分ける
X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
y = data[:, -1]   # ラベル（Accent）

# 特徴量をDataFrameに変換してCSVファイルとして保存
X_df = pd.DataFrame(X, columns=['Mean', 'Max', 'Min', 'Std', 'Median'])
X_df.to_csv('output_features.csv', index=False)  # index=Falseでインデックスを保存しない

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

def load_and_preprocess_data(folder_path):
    # f0_statistics_1.csv から f0_statistics_5000.csv までのファイルを指定して読み込む
    csv_files = [f"{folder_path}/f0_statistics_{i}.csv" for i in range(1, 5001)]
    
    # 音声ごとに特徴量とラベルを保存するリスト
    audio_data = {}

    for file in csv_files:
        # CSVを読み込む
        df = pd.read_csv(file)
        # 必要な列を取得
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        
        # 音声のIDをファイル名から取得
        audio_id = os.path.basename(file).split('.')[0]  # 例: 'f0_statistics_1'
        
        # 音声ごとに特徴量とラベルを保存
        audio_data[audio_id] = data

    return audio_data

# データのロード
audio_data = load_and_preprocess_data('f0_stac_result_5000')

# 保存先のディレクトリを作成
output_folder = 'processed_audio_data'
os.makedirs(output_folder, exist_ok=True)

# 各音声のデータをCSVファイルに保存
for audio_id, data in audio_data.items():
    # 特徴量とラベルを分ける
    X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
    y = data[:, -1]   # ラベル（Accent）
    
    # DataFrameを作成
    df = pd.DataFrame(X, columns=['Mean', 'Max', 'Min', 'Std', 'Median'])
    df['Accent'] = y  # アクセントラベルを追加
    
    # CSVファイルとして保存
    output_file = os.path.join(output_folder, f"{audio_id}.csv")
    df.to_csv(output_file, index=False)

    print(f"Saved: {output_file}")

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# 入力；f0_stac_result_5000
# 出力；prepared_features_dataとして保存
# 特徴量とアクセントの統合データを特徴量だけとアクセントだけのファイルに分けた！
import pandas as pd
import numpy as np
import glob
import os
import torch

# データのロードと整形を行う関数
def load_and_preprocess_data(folder_path):
    # 指定したフォルダー内のCSVファイルを全て取得
    csv_files = [f"{folder_path}/f0_statistics_{i}.csv" for i in range(1, 5001)]
    
    audio_data = {}

    # 各CSVファイルを読み込み
    for file in csv_files:
        # CSVファイルをDataFrameとして読み込み
        df = pd.read_csv(file)
        
        # 特徴量とラベルを抽出
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        
        # ファイル名から音声IDを取得
        audio_id = os.path.basename(file).split('.')[0]  # 例: 'f0_statistics_1'

        # 音声IDをキーとしてデータを辞書に格納
        audio_data[audio_id] = data

    return audio_data

# データのロード
audio_data = load_and_preprocess_data('f0_stac_result_5000')

# 特徴量とラベルに分け、PyTorchのテンソルに変換する関数
def prepare_data(audio_data):
    X_list = []
    y_list = []
    
    for audio_id, data in audio_data.items():
        # 特徴量とラベルを分ける
        X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
        y = data[:, -1]   # ラベル（Accent）

        # 各音声データの特徴量とラベルをリストに追加
        X_list.append(X)
        y_list.append(y)

    # リストをPyTorchのテンソルに変換
    X_tensor = [torch.tensor(X, dtype=torch.float32) for X in X_list]
    y_tensor = [torch.tensor(y, dtype=torch.long) for y in y_list]

    return X_tensor, y_tensor

# 特徴量とラベルを準備する
X_tensor, y_tensor = prepare_data(audio_data)

# データの確認
for i in range(len(X_tensor)):
    print(f"Audio ID: {i + 1}, Feature shape: {X_tensor[i].shape}, Label shape: {y_tensor[i].shape}")

# データを保存する（必要に応じて）
def save_data(X_tensor, y_tensor, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for i, (X, y) in enumerate(zip(X_tensor, y_tensor)):
        # 特徴量とラベルをCSVファイルとして保存
        df_X = pd.DataFrame(X.numpy(), columns=['Mean', 'Max', 'Min', 'Std', 'Median'])
        df_y = pd.DataFrame(y.numpy(), columns=['Accent'])
        
        df_X.to_csv(os.path.join(output_folder, f"features_audio_{i + 1}.csv"), index=False)
        df_y.to_csv(os.path.join(output_folder, f"labels_audio_{i + 1}.csv"), index=False)

# 特徴量とラベルを保存（必要に応じて）
save_data(X_tensor, y_tensor, 'prepared_features_data')

print("データの準備と保存が完了しました。")

In [ ]:
# ここから！
import pandas as pd
import os
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)
print(X_list)
print(y_list)

In [ ]:
# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = split_data(X_list, y_list)

In [ ]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 最後のタイムステップの出力を使う
        return out

# ハイパーパラメータの設定
input_size = 5  # 特徴量の次元数 (Mean, Max, Min, Std, Median の5つ)
hidden_size = 64
output_size = len(set(torch.cat(y_list).numpy()))  # ラベル（アクセント）の数

# モデルのインスタンス化
model = LSTMModel(input_size, hidden_size, output_size)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # LSTMの出力
        out = self.fc(out)  # 全ての時間ステップの出力を使用
        return out

# モデル学習のための関数
def train_model(model, X_train, y_train, num_epochs=20, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        total_loss = 0.0
        for i, (inputs, labels) in enumerate(zip(X_train, y_train)):
            # バッチ化されたデータをモデルに渡して予測を取得
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            
            # 出力とラベルの形を一致させる
            outputs = outputs.view(-1, outputs.size(-1))
            
            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # 各エポックの平均損失を出力
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(X_train):.4f}")

# モデルの初期化（ラベルが0, 1, 2の場合は output_size = 3 とする）
input_size = 5  # 特徴量の数（Mean, Max, Min, Std, Median）
hidden_size = 64  # 隠れ層のユニット数
output_size = 3  # ラベルの範囲が 0, 1, 2 ならば output_size = 3
num_layers = 1  # LSTMのレイヤー数

model = LSTMAccentModel(input_size, hidden_size, output_size)

# データのロード（前に定義した `X_tensor` と `y_tensor` を使用）
X_train = X_tensor  # 特徴量
y_train = y_tensor  # ラベル

# モデルの学習を実行
train_model(model, X_train, y_train)

In [ ]:
# まとめ
import pandas as pd
import os
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"re_features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"re_labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = split_data(X_list, y_list)

import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 最後のタイムステップの出力を使う
        return out

# ハイパーパラメータの設定
input_size = 5  # 特徴量の次元数 (Mean, Max, Min, Std, Median の5つ)
hidden_size = 64
output_size = len(set(torch.cat(y_list).numpy()))  # ラベル（アクセント）の数

# モデルのインスタンス化
model = LSTMModel(input_size, hidden_size, output_size)

import torch
import torch.nn as nn
import torch.optim as optim

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # LSTMの出力
        out = self.fc(out)  # 全ての時間ステップの出力を使用
        return out

# モデル学習のための関数
def train_model(model, X_train, y_train, num_epochs=20, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        total_loss = 0.0
        for i, (inputs, labels) in enumerate(zip(X_train, y_train)):
            # バッチ化されたデータをモデルに渡して予測を取得
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            
            # 出力とラベルの形を一致させる
            outputs = outputs.view(-1, outputs.size(-1))
            
            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # 各エポックの平均損失を出力
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(X_train):.4f}")

# モデルの初期化（ラベルが0, 1, 2の場合は output_size = 3 とする）
input_size = 5  # 特徴量の数（Mean, Max, Min, Std, Median）
hidden_size = 64  # 隠れ層のユニット数
output_size = 3  # ラベルの範囲が 0, 1, 2 ならば output_size = 3
num_layers = 1  # LSTMのレイヤー数

model = LSTMAccentModel(input_size, hidden_size, output_size)

# データのロード（前に定義した `X_tensor` と `y_tensor` を使用）
X_train = X_tensor  # 特徴量
y_train = y_tensor  # ラベル

# モデルの学習を実行
train_model(model, X_train, y_train)

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# 学習過程を描画
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 進捗バー表示のためにtqdmをインポート
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # LSTMの出力
        out = self.fc(out)  # 全ての時間ステップの出力を使用
        return out

# モデル学習のための関数
def train_model(model, X_train, y_train, num_epochs=20, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    losses = []  # 各エポックの損失を記録するリスト

    for epoch in tqdm(range(num_epochs), desc="Training Progress"):  # tqdmで進捗を表示
        total_loss = 0.0
        for i, (inputs, labels) in enumerate(zip(X_train, y_train)):
            # バッチ化されたデータをモデルに渡して予測を取得
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            
            # 出力とラベルの形を一致させる
            outputs = outputs.view(-1, outputs.size(-1))
            
            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 各エポックの平均損失を保存
        avg_loss = total_loss / len(X_train)
        losses.append(avg_loss)

        # 各エポックの平均損失を出力
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 学習の進行状況をプロット
    plt.plot(range(1, num_epochs + 1), losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()

# モデルの初期化（ラベルが0, 1, 2の場合は output_size = 3 とする）
input_size = 5  # 特徴量の数（Mean, Max, Min, Std, Median）
hidden_size = 64  # 隠れ層のユニット数
output_size = 3  # ラベルの範囲が 0, 1, 2 ならば output_size = 3
num_layers = 1  # LSTMのレイヤー数

model = LSTMAccentModel(input_size, hidden_size, output_size)

# データのロード（前に定義した `X_tensor` と `y_tensor` を使用）
X_train = X_tensor  # 特徴量
y_train = y_tensor  # ラベル

# モデルの学習を実行
train_model(model, X_train, y_train)

In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 進捗バー表示のためにtqdmをインポート
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # LSTMの出力
        out = self.fc(out)  # 全ての時間ステップの出力を使用
        return out

# モデル学習のための関数
def train_model(model, X_train, y_train, num_epochs=20, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    losses = []  # 各エポックの損失を記録するリスト

    for epoch in range(num_epochs):
        total_loss = 0.0
        
        # tqdmでエポック全体の進捗バーを表示（全バッチを100%とする）
        with tqdm(total=len(X_train), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
            for i, (inputs, labels) in enumerate(zip(X_train, y_train)):
                # バッチ化されたデータをモデルに渡して予測を取得
                inputs = inputs.unsqueeze(0)  # バッチ次元を追加
                labels = labels.view(-1)  # ラベルを1次元に変換
                
                outputs = model(inputs)
                
                # 出力とラベルの形を一致させる
                outputs = outputs.view(-1, outputs.size(-1))
                
                # 損失計算
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                
                # バックプロパゲーション
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # バッチごとに進捗を更新
                pbar.update(1)

        # 各エポックの平均損失を保存
        avg_loss = total_loss / len(X_train)
        losses.append(avg_loss)

        # 各エポックの平均損失を出力
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 学習の進行状況をプロット
    plt.plot(range(1, num_epochs + 1), losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()

# モデルの初期化（ラベルが0, 1, 2の場合は output_size = 3 とする）
input_size = 5  # 特徴量の数（Mean, Max, Min, Std, Median）
hidden_size = 64  # 隠れ層のユニット数
output_size = 3  # ラベルの範囲が 0, 1, 2 ならば output_size = 3
num_layers = 1  # LSTMのレイヤー数

model = LSTMAccentModel(input_size, hidden_size, output_size)

# データのロード（前に定義した `X_tensor` と `y_tensor` を使用）
X_train = X_tensor  # 特徴量
y_train = y_tensor  # ラベル

# モデルの学習を実行
train_model(model, X_train, y_train)

In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 進捗バー表示のためにtqdmをインポート
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # LSTMの出力
        out = self.fc(out)  # 全ての時間ステップの出力を使用
        return out

# モデル学習のための関数
def train_model(model, X_train, y_train, num_epochs=100, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    losses = []  # 各エポックの損失を記録するリスト

    for epoch in range(num_epochs):
        total_loss = 0.0
        
        # tqdmでエポック全体の進捗バーを表示（全バッチを100%とする）
        with tqdm(total=len(X_train), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
            for i, (inputs, labels) in enumerate(zip(X_train, y_train)):
                # バッチ化されたデータをモデルに渡して予測を取得
                inputs = inputs.unsqueeze(0)  # バッチ次元を追加
                labels = labels.view(-1)  # ラベルを1次元に変換
                
                outputs = model(inputs)
                
                # 出力とラベルの形を一致させる
                outputs = outputs.view(-1, outputs.size(-1))
                
                # 損失計算
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                
                # バックプロパゲーション
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # バッチごとに進捗を更新
                pbar.update(1)

        # 各エポックの平均損失を保存
        avg_loss = total_loss / len(X_train)
        losses.append(avg_loss)

        # 各エポックの平均損失を出力
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 学習の進行状況をプロット
    plt.plot(range(1, num_epochs + 1), losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()

# モデルの初期化（ラベルが0, 1, 2の場合は output_size = 3 とする）
input_size = 5  # 特徴量の数（Mean, Max, Min, Std, Median）
hidden_size = 64  # 隠れ層のユニット数
output_size = 3  # ラベルの範囲が 0, 1, 2 ならば output_size = 3
num_layers = 1  # LSTMのレイヤー数

model = LSTMAccentModel(input_size, hidden_size, output_size)

# データのロード（前に定義した `X_tensor` と `y_tensor` を使用）
X_train = X_tensor  # 特徴量
y_train = y_tensor  # ラベル

# モデルの学習を実行
train_model(model, X_train, y_train)

In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 進捗バー表示のためにtqdmをインポート
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # LSTMの出力
        out = self.fc(out)  # 全ての時間ステップの出力を使用
        return out

# モデル学習のための関数
def train_model(model, X_train, y_train, num_epochs=200, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    losses = []  # 各エポックの損失を記録するリスト

    for epoch in range(num_epochs):
        total_loss = 0.0
        
        # tqdmでエポック全体の進捗バーを表示（全バッチを100%とする）
        with tqdm(total=len(X_train), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
            for i, (inputs, labels) in enumerate(zip(X_train, y_train)):
                # バッチ化されたデータをモデルに渡して予測を取得
                inputs = inputs.unsqueeze(0)  # バッチ次元を追加
                labels = labels.view(-1)  # ラベルを1次元に変換
                
                outputs = model(inputs)
                
                # 出力とラベルの形を一致させる
                outputs = outputs.view(-1, outputs.size(-1))
                
                # 損失計算
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                
                # バックプロパゲーション
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # バッチごとに進捗を更新
                pbar.update(1)

        # 各エポックの平均損失を保存
        avg_loss = total_loss / len(X_train)
        losses.append(avg_loss)

        # 各エポックの平均損失を出力
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 学習の進行状況をプロット
    plt.plot(range(1, num_epochs + 1), losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()

# モデルの初期化（ラベルが0, 1, 2の場合は output_size = 3 とする）
input_size = 5  # 特徴量の数（Mean, Max, Min, Std, Median）
hidden_size = 64  # 隠れ層のユニット数
output_size = 3  # ラベルの範囲が 0, 1, 2 ならば output_size = 3
num_layers = 1  # LSTMのレイヤー数

model = LSTMAccentModel(input_size, hidden_size, output_size)

# データのロード（前に定義した `X_tensor` と `y_tensor` を使用）
X_train = X_tensor  # 特徴量
y_train = y_tensor  # ラベル

# モデルの学習を実行
train_model(model, X_train, y_train)

In [ ]:
# 自己教師あり学習
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 進捗バー表示のためにtqdmをインポート
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    
    # ラベルなしデータの準備
    X_unlabeled = X_test  # 評価用データの一部をラベルなしデータとして利用
    return X_train, y_train, X_unlabeled

# 学習データ、評価データ、ラベルなしデータに分割
X_train, y_train, X_unlabeled = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習用のモデルを定義
class SelfSupervisedModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SelfSupervisedModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)  # 入力と同じ次元に戻す

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習モデルの初期化
self_supervised_model = SelfSupervisedModel(input_size=5, hidden_size=64)  # 特徴量の数に応じて調整

# 自己教師あり学習のトレーニングループの修正
def train_self_supervised_model(model, X_unlabeled, num_epochs=100, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()  # 入力を再構築するための損失関数
    loss_history = []  # 損失を記録するリストを関数内で定義

    for epoch in range(num_epochs):
        total_loss = 0  # エポックごとの累積損失
        for inputs in X_unlabeled:
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            outputs = model(inputs)  # モデルの出力を取得

            # 損失計算
            loss = criterion(outputs, inputs)
            total_loss += loss.item()  # 累積損失を加算
            
            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_unlabeled)  # 平均損失を計算
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Self-supervised Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Self-supervised Learning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# 自己教師あり学習の実行
train_self_supervised_model(self_supervised_model, X_unlabeled)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=64, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=100, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# 自己教師あり学習
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 進捗バー表示のためにtqdmをインポート
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.8, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    
    # ラベルなしデータの準備
    X_unlabeled = X_test  # 評価用データの一部をラベルなしデータとして利用
    return X_train, y_train, X_unlabeled

# 学習データ、評価データ、ラベルなしデータに分割
X_train, y_train, X_unlabeled = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習用のモデルを定義
class SelfSupervisedModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SelfSupervisedModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)  # 入力と同じ次元に戻す

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習モデルの初期化
self_supervised_model = SelfSupervisedModel(input_size=5, hidden_size=64)  # 特徴量の数に応じて調整

# 自己教師あり学習のトレーニングループの修正
def train_self_supervised_model(model, X_unlabeled, num_epochs=100, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()  # 入力を再構築するための損失関数, 平均二乗誤差
    loss_history = []  # 損失を記録するリストを関数内で定義

    for epoch in range(num_epochs):
        total_loss = 0  # エポックごとの累積損失
        for inputs in X_unlabeled:
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            outputs = model(inputs)  # モデルの出力を取得

            # 損失計算
            loss = criterion(outputs, inputs)
            total_loss += loss.item()  # 累積損失を加算
            
            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_unlabeled)  # 平均損失を計算
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Self-supervised Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Self-supervised Learning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# 自己教師あり学習の実行
train_self_supervised_model(self_supervised_model, X_unlabeled)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=64, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=100, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=64, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=100, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=128, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=100, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=256, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=100, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=256, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=120, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=256, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=140, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=256, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=180, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=256, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=200, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=256, output_size=3)  # ラベルが0, 1, 2の場合は output_size = 3 とする

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=300, learning_rate=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []  # 損失を記録するリストを定義

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)  # 平均損失を記録
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 損失のプロット
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# 自己学習用；3000個、ファインチューニング用；1000個、評価用；1000個
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用、評価用、ラベルなし用に分ける関数
def split_data(X_list, y_list, test_size=0.4, random_state=42):
    X_temp, X_test, y_temp, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    X_train, X_unlabeled, y_train, _ = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random_state)
    return X_train, y_train, X_test, y_test, X_unlabeled

# 学習データ、評価データ、ラベルなしデータに分割
X_train, y_train, X_test, y_test, X_unlabeled = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習用のモデルを定義
class SelfSupervisedModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SelfSupervisedModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習モデルの初期化
self_supervised_model = SelfSupervisedModel(input_size=5, hidden_size=64)

# 自己教師あり学習のトレーニングループ
def train_self_supervised_model(model, X_unlabeled, num_epochs=100, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    loss_history = []

    for epoch in range(num_epochs):
        total_loss = 0
        for inputs in X_unlabeled:
            inputs = inputs.unsqueeze(0)
            outputs = model(inputs)

            loss = criterion(outputs, inputs)
            total_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_unlabeled)
        loss_history.append(avg_loss)
        print(f"Self-supervised Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Self-supervised Learning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# 自己教師あり学習の実行
train_self_supervised_model(self_supervised_model, X_unlabeled)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=56, output_size=3)

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=100, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)
            labels = labels.view(-1)
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# 評価関数の定義
def evaluate_model(model, X_test, y_test):
    model.eval()  # 評価モードに切り替え
    predictions = []

    with torch.no_grad():  # 勾配計算を無効に
        for inputs in X_test:
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 2)  # 最も高いスコアを持つクラスを選択
            predictions.append(predicted.view(-1))

    # すべての予測を連結
    predicted_tensor = torch.cat(predictions)
    
    # y_testを連結
    y_test_tensor = torch.cat(y_test)
    
    # 正解率を計算
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted_tensor.numpy())
    return accuracy, y_test_tensor, predicted_tensor

# 評価の実行
accuracy, y_test_tensor, predicted_tensor = evaluate_model(model, X_test, y_test)

# 一致率を出力
print(f'Accuracy on the test set: {accuracy:.4f}')

In [ ]:
# 予測結果の保存
import os
import pandas as pd

# 保存するフォルダの作成
output_dir = 'predicted_accents'
os.makedirs(output_dir, exist_ok=True)

# 評価関数の修正: 予測結果をCSVファイルに保存
def save_predictions_to_csv(model, X_test, y_test, output_dir):
    model.eval()  # 評価モードに切り替え

    with torch.no_grad():  # 勾配計算を無効に
        for i, inputs in enumerate(X_test):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 2)  # 最も高いスコアを持つクラスを選択
            
            # 予測結果をTensorからNumPy配列に変換
            predicted_np = predicted.view(-1).numpy()
            
            # CSVファイルに保存
            file_name = os.path.join(output_dir, f'predicted_accent_{i+1}.csv')
            pd.DataFrame(predicted_np, columns=['Predicted Accent']).to_csv(file_name, index=False)

# 予測結果を保存
save_predictions_to_csv(model, X_test, y_test, output_dir)

print(f"Predicted accents saved in the '{output_dir}' directory.")

In [ ]:
import os
import pandas as pd
import torch

# フォルダのパスを指定
predicted_dir = 'predicted_accents'

# y_testをリストに変換する関数
def load_y_test_tensor(y_test):
    # ここでは、y_testがTensorのリストであると仮定します
    return [tensor.numpy() for tensor in y_test]  # TensorをNumPy配列に変換してリストに追加

# y_testをリスト形式でロード
y_test_list = load_y_test_tensor(y_test)

# 一致数をカウントするための変数を初期化
total_matches = 0
total_samples = 0

# 予測結果をロードして一致をカウント
for i in range(1, 1001):  # predicted_accent_1.csv から predicted_accent_1000.csv まで
    file_name = os.path.join(predicted_dir, f'predicted_accent_{i}.csv')
    
    if os.path.exists(file_name):
        # CSVファイルを読み込む
        predicted_df = pd.read_csv(file_name)
        predicted_array = predicted_df['Predicted Accent'].to_numpy()

        # 一致数をカウント
        if i <= len(y_test_list):  # y_testの範囲内であるか確認
            total_matches += (predicted_array == y_test_list[i - 1]).sum()
            total_samples += len(predicted_array)

# 一致率を計算
if total_samples > 0:
    accuracy = total_matches / total_samples
    print(f'一致数: {total_matches} / 総サンプル数: {total_samples} / 一致率: {accuracy:.4f}')
else:
    print('サンプルが存在しません。')


In [ ]:
import os
import pandas as pd
import torch

# フォルダのパスを指定
predicted_dir = 'predicted_accents'

# y_testをリストに変換する関数
def load_y_test_tensor(y_test):
    return [tensor.numpy() for tensor in y_test]  # TensorをNumPy配列に変換してリストに追加

# y_testをリスト形式でロード
y_test_list = load_y_test_tensor(y_test)

# 一致数をカウントするための変数を初期化
total_matches = 0
total_samples = 0
match_class_1 = 0
match_class_2 = 0

# 予測結果をロードして一致をカウント
for i in range(1, 1001):  # predicted_accent_1.csv から predicted_accent_1000.csv まで
    file_name = os.path.join(predicted_dir, f'predicted_accent_{i}.csv')
    
    if os.path.exists(file_name):
        # CSVファイルを読み込む
        predicted_df = pd.read_csv(file_name)
        predicted_array = predicted_df['Predicted Accent'].to_numpy()

        # 一致数をカウント
        if i <= len(y_test_list):  # y_testの範囲内であるか確認
            total_matches += (predicted_array == y_test_list[i - 1]).sum()
            total_samples += len(predicted_array)

            # クラス1の一致数
            match_class_1 += ((predicted_array == 1) & (y_test_list[i - 1] == 1)).sum()

            # クラス2の一致数
            match_class_2 += ((predicted_array == 2) & (y_test_list[i - 1] == 2)).sum()

# 一致率を計算
if total_samples > 0:
    accuracy = total_matches / total_samples
    print(f'総一致数: {total_matches} / クラス1の一致数: {match_class_1} / クラス2の一致数: {match_class_2} / 総サンプル数: {total_samples} / 一致率: {accuracy:.4f}')
else:
    print('サンプルが存在しません。')


In [ ]:
import os
import pandas as pd
import torch

# フォルダのパスを指定
predicted_dir = 'predicted_accents'

# y_testをリストに変換する関数
def load_y_test_tensor(y_test):
    return [tensor.numpy() for tensor in y_test]  # TensorをNumPy配列に変換してリストに追加

# y_testをリスト形式でロード
y_test_list = load_y_test_tensor(y_test)

# 一致数をカウントするための変数を初期化
total_matches = 0
total_samples = 0
match_class_0 = 0
match_class_1 = 0
match_class_2 = 0

# 各クラスの総数をカウントするための変数を初期化
total_class_0 = 0
total_class_1 = 0
total_class_2 = 0

# 予測結果をロードして一致をカウント
for i in range(1, 1001):  # predicted_accent_1.csv から predicted_accent_1000.csv まで
    file_name = os.path.join(predicted_dir, f'predicted_accent_{i}.csv')
    
    if os.path.exists(file_name):
        # CSVファイルを読み込む
        predicted_df = pd.read_csv(file_name)
        predicted_array = predicted_df['Predicted Accent'].to_numpy()

        # 一致数をカウント
        if i <= len(y_test_list):  # y_testの範囲内であるか確認
            total_matches += (predicted_array == y_test_list[i - 1]).sum()
            total_samples += len(predicted_array)

            # 各クラスの一致数と総数をカウント
            match_class_0 += ((predicted_array == 0) & (y_test_list[i - 1] == 0)).sum()
            match_class_1 += ((predicted_array == 1) & (y_test_list[i - 1] == 1)).sum()
            match_class_2 += ((predicted_array == 2) & (y_test_list[i - 1] == 2)).sum()

            total_class_0 += (y_test_list[i - 1] == 0).sum()
            total_class_1 += (y_test_list[i - 1] == 1).sum()
            total_class_2 += (y_test_list[i - 1] == 2).sum()

# 各クラスの一致率を計算
accuracy_class_0 = match_class_0 / total_class_0 if total_class_0 > 0 else 0
accuracy_class_1 = match_class_1 / total_class_1 if total_class_1 > 0 else 0
accuracy_class_2 = match_class_2 / total_class_2 if total_class_2 > 0 else 0

# 結果を出力
print(f'クラス0の一致数: {match_class_0} / クラス0の総数: {total_class_0} / 一致率: {match_class_0}/{total_class_0} = {accuracy_class_0:.4f}')
print(f'クラス1の一致数: {match_class_1} / クラス1の総数: {total_class_1} / 一致率: {match_class_1}/{total_class_1} = {accuracy_class_1:.4f}')
print(f'クラス2の一致数: {match_class_2} / クラス2の総数: {total_class_2} / 一致率: {match_class_2}/{total_class_2} = {accuracy_class_2:.4f}')


In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# アクセントを高低(1, 0)の2値に変更
# 入力；re_f0_stac_result_5000
# 出力；re_prepared_features_dataとして保存
# 特徴量とアクセントの統合データを特徴量だけとアクセントだけのファイルに分けた！
import pandas as pd
import numpy as np
import glob
import os
import torch

# データのロードと整形を行う関数
def load_and_preprocess_data(folder_path):
    # 指定したフォルダー内のCSVファイルを全て取得
    csv_files = [f"{folder_path}/re_f0_statistics_{i}.csv" for i in range(1, 5001)]
    
    audio_data = {}

    # 各CSVファイルを読み込み
    for file in csv_files:
        # CSVファイルをDataFrameとして読み込み
        df = pd.read_csv(file)
        
        # 特徴量とラベルを抽出
        data = df[['Mean', 'Max', 'Min', 'Std', 'Median', 'Accent']].values
        
        # ファイル名から音声IDを取得
        audio_id = os.path.basename(file).split('.')[0]  # 例: 'f0_statistics_1'

        # 音声IDをキーとしてデータを辞書に格納
        audio_data[audio_id] = data

    return audio_data

# データのロード
audio_data = load_and_preprocess_data('re_f0_stac_result_5000')

# 特徴量とラベルに分け、PyTorchのテンソルに変換する関数
def prepare_data(audio_data):
    X_list = []
    y_list = []
    
    for audio_id, data in audio_data.items():
        # 特徴量とラベルを分ける
        X = data[:, :-1]  # 特徴量（Mean, Max, Min, Std, Median）
        y = data[:, -1]   # ラベル（Accent）

        # 各音声データの特徴量とラベルをリストに追加
        X_list.append(X)
        y_list.append(y)

    # リストをPyTorchのテンソルに変換
    X_tensor = [torch.tensor(X, dtype=torch.float32) for X in X_list]
    y_tensor = [torch.tensor(y, dtype=torch.long) for y in y_list]

    return X_tensor, y_tensor

# 特徴量とラベルを準備する
X_tensor, y_tensor = prepare_data(audio_data)

# データの確認
for i in range(len(X_tensor)):
    print(f"Audio ID: {i + 1}, Feature shape: {X_tensor[i].shape}, Label shape: {y_tensor[i].shape}")

# データを保存する（必要に応じて）
def save_data(X_tensor, y_tensor, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for i, (X, y) in enumerate(zip(X_tensor, y_tensor)):
        # 特徴量とラベルをCSVファイルとして保存
        df_X = pd.DataFrame(X.numpy(), columns=['Mean', 'Max', 'Min', 'Std', 'Median'])
        df_y = pd.DataFrame(y.numpy(), columns=['Accent'])
        
        df_X.to_csv(os.path.join(output_folder, f"re_features_audio_{i + 1}.csv"), index=False)
        df_y.to_csv(os.path.join(output_folder, f"re_labels_audio_{i + 1}.csv"), index=False)

# 特徴量とラベルを保存（必要に応じて）
save_data(X_tensor, y_tensor, 're_prepared_features_data')

print("データの準備と保存が完了しました。")

In [ ]:
# 高低（2→1, 1→0）に変換
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 進捗バー表示のためにtqdmをインポート
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"re_features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"re_labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 're_prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # LSTMの出力
        out = self.fc(out)  # 全ての時間ステップの出力を使用
        return out

# モデル学習のための関数
def train_model(model, X_train, y_train, num_epochs=20, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    losses = []  # 各エポックの損失を記録するリスト

    for epoch in tqdm(range(num_epochs), desc="Training Progress"):  # tqdmで進捗を表示
        total_loss = 0.0
        for i, (inputs, labels) in enumerate(zip(X_train, y_train)):
            # バッチ化されたデータをモデルに渡して予測を取得
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            labels = labels.view(-1)  # ラベルを1次元に変換
            
            outputs = model(inputs)
            
            # 出力とラベルの形を一致させる
            outputs = outputs.view(-1, outputs.size(-1))
            
            # 損失計算
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 各エポックの平均損失を保存
        avg_loss = total_loss / len(X_train)
        losses.append(avg_loss)

        # 各エポックの平均損失を出力
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 学習の進行状況をプロット
    plt.plot(range(1, num_epochs + 1), losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()

# モデルの初期化（ラベルが0, 1の場合は output_size = 2 とする）
input_size = 5  # 特徴量の数（Mean, Max, Min, Std, Median）
hidden_size = 64  # 隠れ層のユニット数
output_size = 2  # ラベルの範囲が 0, 1 ならば output_size = 2
num_layers = 1  # LSTMのレイヤー数

model = LSTMAccentModel(input_size, hidden_size, output_size)

# モデルの学習を実行
train_model(model, X_train, y_train)

In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # 進捗バー表示のためにtqdmをインポート
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        # 特徴量を読み込む
        features_file = os.path.join(folder_path, f"re_features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"re_labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        # DataFrameからnumpy配列に変換し、PyTorchのTensorに変換
        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 're_prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用と評価用に分ける関数
def split_data(X_list, y_list, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = split_data(X_list, y_list)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # LSTMの出力
        out = self.fc(out)  # 全ての時間ステップの出力を使用
        return out

# モデル学習のための関数
def train_model(model, X_train, y_train, num_epochs=100, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    losses = []  # 各エポックの損失を記録するリスト

    for epoch in range(num_epochs):
        total_loss = 0.0
        
        # tqdmでエポック全体の進捗バーを表示（全バッチを100%とする）
        with tqdm(total=len(X_train), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
            for i, (inputs, labels) in enumerate(zip(X_train, y_train)):
                # バッチ化されたデータをモデルに渡して予測を取得
                inputs = inputs.unsqueeze(0)  # バッチ次元を追加
                labels = labels.view(-1)  # ラベルを1次元に変換
                
                outputs = model(inputs)
                
                # 出力とラベルの形を一致させる
                outputs = outputs.view(-1, outputs.size(-1))
                
                # 損失計算
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                
                # バックプロパゲーション
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # バッチごとに進捗を更新
                pbar.update(1)

        # 各エポックの平均損失を保存
        avg_loss = total_loss / len(X_train)
        losses.append(avg_loss)

        # 各エポックの平均損失を出力
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # 学習の進行状況をプロット
    plt.plot(range(1, num_epochs + 1), losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()

# モデルの初期化（ラベルが0, 1の場合は output_size = 2 とする）
input_size = 5  # 特徴量の数（Mean, Max, Min, Std, Median）
hidden_size = 64  # 隠れ層のユニット数
output_size = 2  # ラベルの範囲が 0, 1 ならば output_size = 2
num_layers = 1  # LSTMのレイヤー数

model = LSTMAccentModel(input_size, hidden_size, output_size)

# モデルの学習を実行
train_model(model, X_train, y_train)

In [ ]:
# 学習用：3500, ファインチューニング用：750, 評価用；750
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        features_file = os.path.join(folder_path, f"re_features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"re_labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 're_prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用、ファインチューニング用、評価用に分ける関数
def split_data(X_list, y_list, train_size=0.7, val_size=0.15, random_state=42):
    # 学習用データと残りを分割
    X_train, X_temp, y_train, y_temp = train_test_split(X_list, y_list, test_size=(1 - train_size), random_state=random_state)

    # 残りのデータをファインチューニング用と評価用に分割
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(0.5), random_state=random_state)

    return X_train, y_train, X_val, y_val, X_test, y_test

# 学習データ、ファインチューニングデータ、評価データに分割
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X_list, y_list, train_size=3500/5000, val_size=750/5000)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習用のモデルを定義
class SelfSupervisedModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SelfSupervisedModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習モデルの初期化
self_supervised_model = SelfSupervisedModel(input_size=5, hidden_size=64)

# 自己教師あり学習のトレーニングループ
def train_self_supervised_model(model, X_train, num_epochs=100, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    loss_history = []

    for epoch in range(num_epochs):
        total_loss = 0
        for inputs in X_train:
            inputs = inputs.unsqueeze(0)
            outputs = model(inputs)

            loss = criterion(outputs, inputs)
            total_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)
        print(f"Self-supervised Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Self-supervised Learning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# 自己教師あり学習の実行
train_self_supervised_model(self_supervised_model, X_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=56, output_size=2)

# ファインチューニングの実行
def fine_tune_model(model, X_val, y_val, num_epochs=100, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_val, y_val):
            inputs = inputs.unsqueeze(0)
            labels = labels.view(-1)
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_val)
        loss_history.append(avg_loss)
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_val, y_val)

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# 評価関数の定義
def evaluate_model(model, X_test, y_test):
    model.eval()  # 評価モードに切り替え
    predictions = []

    with torch.no_grad():  # 勾配計算を無効に
        for inputs in X_test:
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 2)  # 最も高いスコアを持つクラスを選択
            predictions.append(predicted.view(-1))

    # すべての予測を連結
    predicted_tensor = torch.cat(predictions)
    
    # y_testを連結
    y_test_tensor = torch.cat(y_test)
    
    # 正解率を計算
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted_tensor.numpy())
    return accuracy, y_test_tensor, predicted_tensor

# 評価の実行
accuracy, y_test_tensor, predicted_tensor = evaluate_model(model, X_test, y_test)

# 一致率を出力
print(f'Accuracy on the test set: {accuracy:.4f}')

In [ ]:
# 予測結果の保存
import os
import pandas as pd

# 保存するフォルダの作成
output_dir = 're_predicted_accents'
os.makedirs(output_dir, exist_ok=True)

# 評価関数の修正: 予測結果をCSVファイルに保存
def save_predictions_to_csv(model, X_test, y_test, output_dir):
    model.eval()  # 評価モードに切り替え

    with torch.no_grad():  # 勾配計算を無効に
        for i, inputs in enumerate(X_test):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 2)  # 最も高いスコアを持つクラスを選択
            
            # 予測結果をTensorからNumPy配列に変換
            predicted_np = predicted.view(-1).numpy()
            
            # CSVファイルに保存
            file_name = os.path.join(output_dir, f're_predicted_accent_{i+1}.csv')
            pd.DataFrame(predicted_np, columns=['Predicted Accent']).to_csv(file_name, index=False)

# 予測結果を保存
save_predictions_to_csv(model, X_test, y_test, output_dir)

print(f"Predicted accents saved in the '{output_dir}' directory.")

In [ ]:
import os
import pandas as pd
import torch

# フォルダのパスを指定
predicted_dir = 're_predicted_accents'

# y_testをリストに変換する関数
def load_y_test_tensor(y_test):
    return [tensor.numpy() for tensor in y_test]  # TensorをNumPy配列に変換してリストに追加

# y_testをリスト形式でロード
y_test_list = load_y_test_tensor(y_test)

# 一致数をカウントするための変数を初期化
total_matches = 0
total_samples = 0
match_class_0 = 0
match_class_1 = 0

# 各クラスの総数をカウントするための変数を初期化
total_class_0 = 0
total_class_1 = 0

# 予測結果をロードして一致をカウント
for i in range(1, 1001):  # predicted_accent_1.csv から predicted_accent_1000.csv まで
    file_name = os.path.join(predicted_dir, f're_predicted_accent_{i}.csv')
    
    if os.path.exists(file_name):
        # CSVファイルを読み込む
        predicted_df = pd.read_csv(file_name)
        predicted_array = predicted_df['Predicted Accent'].to_numpy()

        # 一致数をカウント
        if i <= len(y_test_list):  # y_testの範囲内であるか確認
            total_matches += (predicted_array == y_test_list[i - 1]).sum()
            total_samples += len(predicted_array)

            # クラス0とクラス1の一致数と総数をカウント
            match_class_0 += ((predicted_array == 0) & (y_test_list[i - 1] == 0)).sum()
            match_class_1 += ((predicted_array == 1) & (y_test_list[i - 1] == 1)).sum()

            total_class_0 += (y_test_list[i - 1] == 0).sum()
            total_class_1 += (y_test_list[i - 1] == 1).sum()

# 各クラスの一致率を計算
accuracy_class_0 = match_class_0 / total_class_0 if total_class_0 > 0 else 0
accuracy_class_1 = match_class_1 / total_class_1 if total_class_1 > 0 else 0

# 結果を出力
print(f'(下降)クラス0の一致数: {match_class_0} / クラス0の総数: {total_class_0} / 一致率: {match_class_0}/{total_class_0} = {accuracy_class_0:.4f}')
print(f'(上昇)クラス1の一致数: {match_class_1} / クラス1の総数: {total_class_1} / 一致率: {match_class_1}/{total_class_1} = {accuracy_class_1:.4f}')

In [ ]:
# 再実験(0, 1, 2)
# 学習用：3500, ファインチューニング用：750, 評価用；750
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# データをロードする関数
def load_csv_data(folder_path, num_files=5000):
    X_list = []
    y_list = []

    for i in range(1, num_files + 1):
        features_file = os.path.join(folder_path, f"features_audio_{i}.csv")
        labels_file = os.path.join(folder_path, f"labels_audio_{i}.csv")

        df_X = pd.read_csv(features_file)
        df_y = pd.read_csv(labels_file)

        X = torch.tensor(df_X.values, dtype=torch.float32)
        y = torch.tensor(df_y.values.flatten(), dtype=torch.long)

        X_list.append(X)
        y_list.append(y)

    return X_list, y_list

# 特徴量とラベルのロード
folder_path = 'prepared_features_data'
X_list, y_list = load_csv_data(folder_path)

# データを学習用、ファインチューニング用、評価用に分ける関数
def split_data(X_list, y_list, train_size=0.7, val_size=0.15, random_state=42):
    # 学習用データと残りを分割
    X_train, X_temp, y_train, y_temp = train_test_split(X_list, y_list, test_size=(1 - train_size), random_state=random_state)

    # 残りのデータをファインチューニング用と評価用に分割
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(0.5), random_state=random_state)

    return X_train, y_train, X_val, y_val, X_test, y_test

# 学習データ、ファインチューニングデータ、評価データに分割
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X_list, y_list, train_size=3500/5000, val_size=750/5000)

# LSTMモデル定義
class LSTMAccentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMAccentModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習用のモデルを定義
class SelfSupervisedModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SelfSupervisedModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 自己教師あり学習モデルの初期化
self_supervised_model = SelfSupervisedModel(input_size=5, hidden_size=64)

# 自己教師あり学習のトレーニングループ
def train_self_supervised_model(model, X_train, num_epochs=100, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    loss_history = []

    for epoch in range(num_epochs):
        total_loss = 0
        for inputs in X_train:
            inputs = inputs.unsqueeze(0)
            outputs = model(inputs)

            loss = criterion(outputs, inputs)
            total_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)
        print(f"Self-supervised Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Self-supervised Learning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# 自己教師あり学習の実行
train_self_supervised_model(self_supervised_model, X_train)

In [ ]:
# ファインチューニングのためのLSTMモデルの初期化
model = LSTMAccentModel(input_size=5, hidden_size=56, output_size=3)

# ファインチューニングの実行
def fine_tune_model(model, X_train, y_train, num_epochs=100, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_history = []

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in zip(X_train, y_train):
            inputs = inputs.unsqueeze(0)
            labels = labels.view(-1)
            
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(X_train)
        loss_history.append(avg_loss)
        print(f"Fine-tuning Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
    plt.title('Fine-tuning Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.show()

# ファインチューニングの実行
fine_tune_model(model, X_train, y_train)

In [ ]:
# 予測結果の保存
import os
import pandas as pd

# 保存するフォルダの作成
output_dir = 'predicted_accents'
os.makedirs(output_dir, exist_ok=True)

# 評価関数の修正: 予測結果をCSVファイルに保存
def save_predictions_to_csv(model, X_test, y_test, output_dir):
    model.eval()  # 評価モードに切り替え

    with torch.no_grad():  # 勾配計算を無効に
        for i, inputs in enumerate(X_test):
            inputs = inputs.unsqueeze(0)  # バッチ次元を追加
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 2)  # 最も高いスコアを持つクラスを選択
            
            # 予測結果をTensorからNumPy配列に変換
            predicted_np = predicted.view(-1).numpy()
            
            # CSVファイルに保存
            file_name = os.path.join(output_dir, f'predicted_accent_{i+1}.csv')
            pd.DataFrame(predicted_np, columns=['Predicted Accent']).to_csv(file_name, index=False)

# 予測結果を保存
save_predictions_to_csv(model, X_test, y_test, output_dir)

print(f"Predicted accents saved in the '{output_dir}' directory.")

In [ ]:
import os
import pandas as pd
import torch

# フォルダのパスを指定
predicted_dir = 'predicted_accents'

# y_testをリストに変換する関数
def load_y_test_tensor(y_test):
    return [tensor.numpy() for tensor in y_test]  # TensorをNumPy配列に変換してリストに追加

# y_testをリスト形式でロード
y_test_list = load_y_test_tensor(y_test)

# 一致数をカウントするための変数を初期化
total_matches = 0
total_samples = 0
match_class_0 = 0
match_class_1 = 0
match_class_2 = 0

# 各クラスの総数をカウントするための変数を初期化
total_class_0 = 0
total_class_1 = 0
total_class_2 = 0

# 予測結果をロードして一致をカウント
for i in range(1, 1001):  # predicted_accent_1.csv から predicted_accent_1000.csv まで
    file_name = os.path.join(predicted_dir, f'predicted_accent_{i}.csv')
    
    if os.path.exists(file_name):
        # CSVファイルを読み込む
        predicted_df = pd.read_csv(file_name)
        predicted_array = predicted_df['Predicted Accent'].to_numpy()

        # 一致数をカウント
        if i <= len(y_test_list):  # y_testの範囲内であるか確認
            total_matches += (predicted_array == y_test_list[i - 1]).sum()
            total_samples += len(predicted_array)

            # 各クラスの一致数と総数をカウント
            match_class_0 += ((predicted_array == 0) & (y_test_list[i - 1] == 0)).sum()
            match_class_1 += ((predicted_array == 1) & (y_test_list[i - 1] == 1)).sum()
            match_class_2 += ((predicted_array == 2) & (y_test_list[i - 1] == 2)).sum()

            total_class_0 += (y_test_list[i - 1] == 0).sum()
            total_class_1 += (y_test_list[i - 1] == 1).sum()
            total_class_2 += (y_test_list[i - 1] == 2).sum()

# 各クラスの一致率を計算
accuracy_class_0 = match_class_0 / total_class_0 if total_class_0 > 0 else 0
accuracy_class_1 = match_class_1 / total_class_1 if total_class_1 > 0 else 0
accuracy_class_2 = match_class_2 / total_class_2 if total_class_2 > 0 else 0

# 結果を出力
print(f'クラス0の一致数: {match_class_0} / クラス0の総数: {total_class_0} / 一致率: {match_class_0}/{total_class_0} = {accuracy_class_0:.4f}')
print(f'クラス1の一致数: {match_class_1} / クラス1の総数: {total_class_1} / 一致率: {match_class_1}/{total_class_1} = {accuracy_class_1:.4f}')
print(f'クラス2の一致数: {match_class_2} / クラス2の総数: {total_class_2} / 一致率: {match_class_2}/{total_class_2} = {accuracy_class_2:.4f}')